In [1]:
import cv2 # type: ignore
import os
import tensorflow as tf # type: ignore
from tensorflow import keras # type: ignore
from PIL import Image # type: ignore
import numpy as np # type: ignore
from sklearn.model_selection import train_test_split # type: ignore
from keras.utils import normalize # type: ignore
from keras.models import Sequential # type: ignore
from keras.layers import Conv2D, MaxPooling2D # type: ignore
from keras.layers import Activation, Dropout, Flatten, Dense # type: ignore
from keras.utils import to_categorical # type: ignore

In [2]:
no_tumor_images = os.listdir('datasets/no/')
yes_tumor_images = os.listdir('datasets/yes/')
dataset = []
label = []

In [3]:
INPUT_SIZE = 64

def process_images(image_list, label_value, folder):
    for image_name in image_list:
        if image_name.endswith('.jpg'):
            image = cv2.imread(f'datasets/{folder}/' + image_name)
            image = Image.fromarray(image, 'RGB')
            image = image.resize((INPUT_SIZE, INPUT_SIZE))
            dataset.append(np.array(image))
            label.append(label_value)

process_images(no_tumor_images, 0, 'no')
process_images(yes_tumor_images, 1, 'yes')

dataset = np.array(dataset)
label = np.array(label)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(dataset, label, test_size=0.3, random_state=0)

x_train = normalize(x_train, axis=1) 
x_test = normalize(x_test, axis=1)

y_train = to_categorical(y_train, num_classes=2) #one-hot encoding
y_test = to_categorical(y_test, num_classes=2)

In [5]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(INPUT_SIZE, INPUT_SIZE, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [6]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=16,
          verbose=1, epochs=25,
          validation_data=(x_test, y_test),
          shuffle=False)

model.save('BrainTumor10EpochsCategorical.h5')


Epoch 1/25


8/8 [==============================] - 2s 46ms/step - loss: 0.6814 - accuracy: 0.5714 - val_loss: 0.7138 - val_accuracy: 0.4615
Epoch 2/25
8/8 [==============================] - 0s 19ms/step - loss: 0.7030 - accuracy: 0.5294 - val_loss: 0.6754 - val_accuracy: 0.5192
Epoch 3/25
8/8 [==============================] - 0s 18ms/step - loss: 0.6782 - accuracy: 0.5462 - val_loss: 0.6700 - val_accuracy: 0.5769
Epoch 4/25
8/8 [==============================] - 0s 18ms/step - loss: 0.6677 - accuracy: 0.5546 - val_loss: 0.6690 - val_accuracy: 0.4615
Epoch 5/25
8/8 [==============================] - 0s 18ms/step - loss: 0.6671 - accuracy: 0.5546 - val_loss: 0.6529 - val_accuracy: 0.5385
Epoch 6/25
8/8 [==============================] - 0s 30ms/step - loss: 0.6420 - accuracy: 0.6134 - val_loss: 0.6407 - val_accuracy: 0.5385
Epoch 7/25
8/8 [==============================] - 0s 19ms/step - loss: 0.6345 - accuracy: 0.5966 - val_loss: 0.6111 - val_accuracy: 0.7692
Epoch 8/25
8/8 [========

c:\Users\kamal\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
evalu = model.evaluate(x_test, y_test)
print(f'Test accuracy: {int(evalu[1]*100)}%')

2/2 [==============================] - 0s 10ms/step - loss: 0.4059 - accuracy: 0.8077
Test accuracy: 80%
